In [ ]:
from pylab import *
%matplotlib inline
caffe_root = '/home/deep/develop/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

# run scripts from caffe root
import os
os.chdir(caffe_root)
# Download data
!data/mnist/get_mnist.sh
# Prepare data
!examples/mnist/create_mnist.sh
# back to examples
os.chdir('examples')

from caffe import layers as L, params as P

def lenet(lmdb, batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.fc1 =   L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.fc1, in_place=True)
    n.score = L.InnerProduct(n.relu1, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    return n.to_proto()
    
with open('/home/deep/Documents/zongzuo/pythoncodes/Caffe tests/lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet('mnist/mnist_train_lmdb', 64)))
    
with open('/home/deep/Documents/zongzuo/pythoncodes/Caffe tests/lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet('mnist/mnist_test_lmdb', 100)))

caffe.set_device(0)
caffe.set_mode_gpu()

### load the solver and create train and test nets
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.SGDSolver('lenet_auto_solver.prototxt')

Downloading...
Creating lmdb...
I1114 22:34:49.331987 23233 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_train_lmdb
I1114 22:34:49.332171 23233 convert_mnist_data.cpp:88] A total of 60000 items.
I1114 22:34:49.332195 23233 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I1114 22:34:53.599746 23233 convert_mnist_data.cpp:108] Processed 60000 files.
I1114 22:34:53.634827 23234 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_test_lmdb
I1114 22:34:53.635382 23234 convert_mnist_data.cpp:88] A total of 10000 items.
I1114 22:34:53.635424 23234 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I1114 22:34:54.291183 23234 convert_mnist_data.cpp:108] Processed 10000 files.
Done.


In [2]:
# each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

[('data', (64, 1, 28, 28)),
 ('label', (64,)),
 ('conv1', (64, 20, 24, 24)),
 ('pool1', (64, 20, 12, 12)),
 ('conv2', (64, 50, 8, 8)),
 ('pool2', (64, 50, 4, 4)),
 ('fc1', (64, 500)),
 ('score', (64, 10)),
 ('loss', ())]